<a href="https://colab.research.google.com/github/jessica8403/TIL/blob/master/text%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [7]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import os, shutil

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets

In [5]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.5 MB/s 
     |████████████████████████████████| 448 kB 23.2 MB/s 


In [ ]:
%%shell
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip install JPype1
pip install konlpy

In [16]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [ ]:
%%shell
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip install /tmp/mecab-python-0.996

In [1]:
from konlpy.tag import Kkma, Komoran, Okt, Hannanum, Mecab

In [3]:
# 교재에서는 실습을 위해서
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [5]:
# 토크나이저 객체 생성(가장 빈도가 높은 단어 1,000개만 선택해서 토크나이징)
tokenizer = Tokenizer(num_words=1000)

# 단어 인덱스 구축
tokenizer.fit_on_texts(samples)

# 인덱스 리트스 확인
sequence = tokenizer.texts_to_sequences(samples)
print(sequence)

# 변환된 숫자 인덱스를 벡터 형태로 변환()
one_hot_encoded = tokenizer.texts_to_matrix(samples, mode='binary')
print(one_hot_encoded)

[[1, 2, 3, 4, 1, 5], [1, 6, 7, 8, 9]]
[[0. 1. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [2]:
kkma = Kkma()
komoran = Komoran()
han = Hannanum()
mecab = Mecab()
okt = Okt()

In [15]:
samples = ['아버지가방에들어가신다', '아기다리고기다리던데이터']

In [16]:
kkma.pos(samples[0])

[('아버지', 'NNG'),
 ('가방', 'NNG'),
 ('에', 'JKM'),
 ('들어가', 'VV'),
 ('시', 'EPH'),
 ('ㄴ다', 'EFN')]

In [17]:
kkma.morphs( samples[1] )

['아', '아', '기다리', '고', '기다리', '던데', '이', '터']

In [18]:
kkma.nouns( samples[0] )

['아버지', '아버지가방', '가방']

In [19]:
mecab.pos( samples[0] )

[('아버지', 'NNG'),
 ('가', 'JKS'),
 ('방', 'NNG'),
 ('에', 'JKB'),
 ('들어가', 'VV'),
 ('신다', 'EP+EC')]

In [20]:
mecab.morphs( samples[1] )

['아기', '다리', '고', '기다리', '던', '데이터']

In [21]:
mecab.nouns( samples[1] )

['아기', '데이터']

In [22]:
komoran.pos( samples[1] )

[('아', 'IC'),
 ('기다리', 'VV'),
 ('고', 'EC'),
 ('기다리', 'VV'),
 ('던', 'ETM'),
 ('데이터', 'NNG')]

In [23]:
samples_to_word = []
for sample in samples:
  # print( mecab.morphs(sample) )
  samples_to_word.append( mecab.morphs(sample) )

samples_to_word

[['아버지', '가', '방', '에', '들어가', '신다'], ['아기', '다리', '고', '기다리', '던', '데이터']]

In [24]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [25]:
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(samples_to_word)
sequence = tokenizer.texts_to_sequences(samples_to_word)
print(sequence) 

one_hot_encoded = tokenizer.texts_to_matrix(samples_to_word, mode='binary')
print(one_hot_encoded)

[[1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12]]
[[0. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [26]:
from tensorflow.keras.datasets import imdb

In [27]:
# 특성으로 사용할 단어의 갯수
max_features = 10000
# 사용할 텍스트의 길이
maxLen = 20

# 정수 리스트로 데이터를 로드
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = max_features)

17473536/17464789 [==============================] - 0s 0us/step
